In [1]:
import tensorflow as tf
import codecs
import numpy as np

In [2]:
data_file = 'data/quijote.txt' # Quijote file in plain text

# Reading "El quijote" with the codecs
data = codecs.open(data_file, 'rU', 'utf-8').read() 
data = data [300000:700000]
chars = list(set(data))
data_size, vocab = len(data), len(chars)
print 'Hay {} caracteres, {} unicos.'.format(data_size, vocab)

# Mini methods to change from character to index and the opposite
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
seq_length = 10 # number of steps to unroll the RNN

Hay 400000 caracteres, 77 unicos.


In [3]:

n, p = 0, 0
n_max = 100000
inputs = []
targets = []

while n <= n_max:
    
    if p + seq_length + 1 >= len(data) or n==0:
        p = 0
    input = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    target = [char_to_ix[ch] for ch in data[p+seq_length:p+seq_length+1]]
    p +=1
    n +=1
    inputs.append(input)
    targets.append(target)

In [4]:
# Target to one hot
one_hot_targets = []
for target in targets:
    _target = target[0]
    one_hot_target = np.zeros(vocab)
    one_hot_target[_target] = 1
    one_hot_targets.append(one_hot_target)
targets = np.array(one_hot_targets)

In [5]:
#Inputs to one hot
one_hot_inputs = []
for input in inputs:
    one_hot_input = []
    for value in input:
        _input = np.zeros(vocab)
        _input[value] = 1
        one_hot_input.append(_input)
    one_hot_input = np.array(one_hot_input)
    one_hot_inputs.append(one_hot_input)
inputs = np.array(one_hot_inputs)

In [6]:
input_data = np.array(inputs)
output_data = np.array(targets)

print input_data.shape
print output_data.shape

(100001, 10, 77)
(100001, 77)


In [7]:
#Data split

LIMIT = 24000
X_test = input_data[LIMIT:]
Y_test = output_data[LIMIT:]

X_train = input_data[:LIMIT]
Y_train = output_data[:LIMIT]

In [8]:
#Network parameters

data = tf.placeholder(tf.float32,[None,seq_length,vocab])
target = tf.placeholder(tf.float32,[None,vocab])

hidden = 2
neurons = 400

cell = tf.contrib.rnn.LSTMCell(neurons, state_is_tuple=True)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([cell] * hidden, state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(stacked_lstm, data, dtype=tf.float32)

val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([neurons, int(target.get_shape()[1])]), name="asfsd")
bias = tf.Variable(tf.constant(0.0, shape=[target.get_shape()[1]]))

In [9]:
y = tf.matmul(last, weight) + bias
prediction = tf.nn.softmax(y)

In [10]:
# Functions definition 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target, logits=y))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
minimize = optimizer.minimize(loss)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(target, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

/Users/diegoperezsastre/virtualenvs/tensorflow/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [12]:
batch_size = 400
no_of_batches = int(len(X_train) / batch_size)
epochs = 4000

print len(X_train),no_of_batches
loss_train_array = []
train_accuracy_array = []
test_accuracy_array = []

24000 60


In [17]:
%%time
with sess.as_default():
    for i in range(epochs):
        ptr = 0
        for j in range(no_of_batches):
            inp, out = X_train[ptr:ptr + batch_size], Y_train[ptr:ptr + batch_size]
            ptr += batch_size
            _, loss_train = sess.run([minimize, loss], {data: inp, target: out})

        if i % 100 == 0:
            train_accuracy = sess.run(
                accuracy, feed_dict={data: X_train, target: Y_train})
            train_accuracy_array.append(train_accuracy)

            test_accuracy = sess.run(
                accuracy, feed_dict={data: X_test, target: Y_test})
            test_accuracy_array.append(test_accuracy)

            print (
                'Epoch %04d, '
                'loss train %.8f, train accuracy %.8f, test accuracy %.8f'
                %
                (i,
                 loss_train,
                 train_accuracy,
                 test_accuracy))

Epoch 0000, loss train 3.11805105, train accuracy 0.16912501, test accuracy 0.16786622
Epoch 0100, loss train 0.19814083, train accuracy 0.93937498, test accuracy 0.39573163
Epoch 0200, loss train 0.06234388, train accuracy 0.97750002, test accuracy 0.40011317
Epoch 0300, loss train 0.16945450, train accuracy 0.95775002, test accuracy 0.38766596
Epoch 0400, loss train 0.10664427, train accuracy 0.97154164, test accuracy 0.39063960
Epoch 0500, loss train 0.07879021, train accuracy 0.97662503, test accuracy 0.38891593
Epoch 0600, loss train 0.20835577, train accuracy 0.95674998, test accuracy 0.38917908
Epoch 0700, loss train 0.13127387, train accuracy 0.94575000, test accuracy 0.37687662
Epoch 0800, loss train 0.14614144, train accuracy 0.95291668, test accuracy 0.38348180
Epoch 0900, loss train 0.14749897, train accuracy 0.94929165, test accuracy 0.38194233
Epoch 1000, loss train 0.10761897, train accuracy 0.96499997, test accuracy 0.38192919
Epoch 1100, loss train 0.12344006, train ac

In [18]:
# sample from the model now and then
preds = []
ix = X_train[:1]

for i in range(1000):
    pred = sess.run(prediction ,{data: ix.reshape(1, seq_length, vocab)})
    index = np.argmax(pred, 1)
    
    preds.append(ix_to_char[index[0]])
    
    zero_array = np.zeros(vocab)
    zero_array[index] = 1
    
    tail = np.append(np.array(ix[0][1:]), np.array(zero_array.reshape(1, vocab)))
    
    ix = tail.reshape(1, seq_length, vocab)

In [19]:
print("".join(preds))

raede, porque los
demás dicirá menestertan el aquelle perdado de reyodo el in es on tiene bien merecido. Bien es verdad
que yo soy hijotes, la quito el rucindados dica que los trueques, si es que tienes dellos.
-Y que cien cacbbre.
-Bespondió el galeote-, que yo tengo la receta de le pasaba del pecho; el cual, grado esto de las mercedes. ver, primerad malencio, y los estu caballos y dejarse ir por donde la vente la pada guar a se despedieme su pena de Paría temo tiene.

Y aquí dio un sospiro, y le pasto de hombre de venerable rostro por se pareben presondé? De la cencalera, y
no dar esposas avánerizo y de la Meigrre la pauendo puso a todos mis cinco
sentidos de ser fengrundo, con la mesma cofradía. Pues, ¿qué será cuando me me mano pevarle que en meyor eso caeleventu, pidi al pasado con alta ni el cubar en ellosa mis, viata la intencia. De las mías entre vengan de los llevan, er algunos el caballero es
genta, pero sinote de torar el de por la reja su señor a se eso. ¿udesa, decir a que

In [26]:
# sample from the model now and then
preds = []
ix = X_test[144:145]

for i in range(1000):
    pred = sess.run(prediction ,{data: ix.reshape(1, seq_length, vocab)})
    index = np.argmax(pred, 1)
    
    preds.append(ix_to_char[index[0]])
    
    zero_array = np.zeros(vocab)
    zero_array[index] = 1
    
    tail = np.append(np.array(ix[0][1:]), np.array(zero_array.reshape(1, vocab)))
    
    ix = tail.reshape(1, seq_length, vocab)

In [24]:
print("".join(preds))

io: de todo el cuerpo, con gura tierta vencedo en pago de sus porque pena pongo detonda en buera en aHta, temgas, y tona y era ne el ente lespado a caballería quitarles los pon de no pudiero este le antes en
la causa de su desgraci. palta, tengon ino con tuvolás caballe años. De lagaldo, como Der tare con el manifes de tode una
doncella que no puede caballo, con
escopetas de rueda, y allí a la gente be no se pues, salenca por te venir a ver en el sino en bueno el rucio!

-Nunca yo acostunte ha tras sí la de saleve tienera con ellos la péndola del este camente eñ reiso tengo lo sía con esta entre el por vuescleros a pla ligen pare traen la quite -dijo don Quijote a mentaca tengo pigoa por tormento a las mil tras mía ser luevo se le saltiera en tus y y por le sabe de los lora cabala porevir a una ma pasanta
por camino.

-Y yo lo entiendo tuento estáve en ellos y ella pencada. Pues, peve entonos ve el rey, cuanto que venclao ve no dices balla el cabante como verces y grave tamente me caer

In [28]:
saver = tf.train.Saver()
save_path = saver.save(sess, "./quijote_model.ckpt")
print("Model saved in file: %s" % save_path)

Model saved in file: ./quijote_model.ckpt


In [14]:
init_op = tf.global_variables_initializer()
